In [12]:
import pandas as pd
import openpyxl
import gensim
from gensim import corpora
from pprint import pprint
import unicodedata as ud
import pymorphy2
from gensim import models
from gensim.utils import simple_preprocess

from gensim.models import Word2Vec
import gensim.models.keyedvectors as word2vec
from gensim.models.phrases import Phrases
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

import numpy as np
from os import listdir

# spacy for lemmatization
import spacy

import matplotlib.pyplot as plt
import requests

import warnings
warnings.filterwarnings('ignore')

In [13]:
## Read data
df_raw = pd.read_excel('elastic_lang.xlsx')
df_raw['col'] = df_raw['cyrillic'].str.lower()
df_lower = df_raw

In [14]:
## PreProcess data (drop unused columns, reindex)
nan_value = float("NaN")
df_lower.replace("", nan_value, inplace=True)

df_lower.dropna(subset = ["cyrillic"], inplace=True)
df_lower = df_lower.drop(columns=['Unnamed: 0'])
df_lower = df_lower.reset_index(drop = True)
df_lower['cyrillic'] = df_lower['cyrillic'].str.lower()
texts = df_lower['cyrillic'].to_list()

In [15]:
## Normalizing tokens
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor\

df_lower['cyrillic'] = df_lower['cyrillic'].str.lstrip()
corpus = df_lower['cyrillic'].str.cat(sep = ' ')

def normalize_tokens(tokens):
    l = len(tokens)
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(tokens[i])[0].normal_form for i in tqdm(range(l))]


data_raw_next = []

data_raw_next = normalize_tokens(corpus.split(sep=' '))

100%|████████████████████████████████████████████████████████████████████████| 996929/996929 [02:52<00:00, 5782.24it/s]


In [ ]:
import pickle
with open("df_lower_cyr_normalized.pkl", "wb") as fp:   #Pickling
    pickle.dump(data_raw_next, fp) 

In [16]:
## Add to df_lower normalized names

df_lower['num_of_cyr'] = ''
df_lower['tokenized_cyr'] = ''
df_lower['num_of_cyr'] = [len(doc.split(sep = ' ')) for doc in df_lower['cyrillic']]
k=0
tmp_list = []
for num in df_lower['num_of_cyr']:
    tmp_list.append(' '.join(data_raw_next[k:(k + int(num))]))
    k+=num
df_lower['tokenized_cyr'] = tmp_list

In [17]:
## Vectorizing normalized names

w2v_model = Word2Vec(
        min_count=12,
        vector_size=10,
        window=2,
        negative=10,
        alpha=0.03,
        min_alpha=0.0007,
        sample=6e-5,
        sg=1)

data = [doc.split() for doc in df_lower['tokenized_cyr']]
    
print('threads ok')
w2v_model.build_vocab(data)
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

w2v_model.init_sims(replace=True)

threads ok


In [18]:
## Make average vector from number of word vectors

def normalize_token(token):
    morph = pymorphy2.MorphAnalyzer()
    return morph.parse(token.lower())[0].normal_form 


def mean_for_vecs(sentence):
    if len(sentence.split(sep = ' ')) > 1:
        text_vector = np.mean([w2v_model.wv[word] for word in sentence.split(sep = ' ')], axis=0)
    else:
        text_vector = w2v_model.wv[word]
    return text_vector

df_mean_vec = pd.DataFrame(columns = ['sentence', 'vec'])
sentences =[]
df_lower = df_lower.reset_index(drop=True)
vecs = []
for i in range(len(df_lower['tokenized_cyr'])):
#     if df_lower.col[i]:
    try:
        vecs.append(mean_for_vecs(df_lower['tokenized_cyr'][i]))
        sentences.append(df_lower['tokenized_cyr'][i])
    except Exception as e :
        pass

df_mean_vec['sentence'] = sentences
df_mean_vec['vec'] = vecs

In [25]:
np.array(list(df_mean_vec['vec']))

array([[ 0.27409828,  0.48373708,  0.4384637 , ..., -0.36254954,
        -0.16276667,  0.4101392 ],
       [-0.12132712,  0.10477108,  0.23433994, ..., -0.13063593,
        -0.2860515 ,  0.07591018],
       [-0.12100955,  0.00312524,  0.19290012, ..., -0.22562097,
        -0.43487152, -0.36587086],
       ...,
       [-0.26895905,  0.39840204,  0.3154568 , ...,  0.03128564,
        -0.21732116, -0.14214496],
       [ 0.0863111 ,  0.0695575 ,  0.34955055, ...,  0.20414725,
        -0.367661  , -0.07540943],
       [ 0.07225259,  0.35874903,  0.37715837, ..., -0.18512839,
        -0.2612232 , -0.16917929]], dtype=float32)

In [26]:
## Model of Clustering


X = np.array(list(df_mean_vec['vec']))

# from nltk.cluster import KMeansClusterer
# import nltk
# NUM_CLUSTERS=200
# kclusterer = KMeansClusterer(NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance, repeats=25, avoid_empty_clusters = True)
# assigned_clusters = kclusterer.cluster(X, assign_clusters=True)


from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=150, init='k-means++', n_init=10, max_iter=50, tol=0.0001, verbose=0, random_state=None, copy_x=True, algorithm='auto').fit_predict(X)


words = df_mean_vec['sentence']
cluster_dict = {}
for i, word in enumerate(words):  
    cluster_dict[str(word)] = str(assigned_clusters[i])

df_clusters = pd.DataFrame({'word': words, 'cluster': assigned_clusters})
df_clusters.to_excel('tmp_df.xlsx')

NameError: name 'assigned_clusters' is not defined

In [30]:
cluster_dict = {}
for i, word in enumerate(words):  
    cluster_dict[str(word)] = str(kmeans[i])

df_clusters = pd.DataFrame({'word': words, 'cluster': kmeans})
df_clusters.to_excel('tmp_df.xlsx')

In [ ]:
cluster_dict = {}
for i, word in enumerate(words):  
    cluster_dict[str(word)] = str(assigned_clusters[i])

In [ ]:
df_clusters = pd.DataFrame({'word': words, 'cluster': assigned_clusters})

In [ ]:
df_clusters.to_excel('tmp_df.xlsx')

In [ ]:
kmeans = cluster.KMeans(n_clusters=100)
kmeans.fit(X)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for inputted data")
print (labels)
print ("Centroids data")
print (centroids)
 
print ("Score (Opposite of the value of X on the K-means objective which is Sum of distances of samples to their closest cluster center):")
print (kmeans.score(X))
 
silhouette_score = metrics.silhouette_score(X, labels, metric='euclidean')
 
print ("Silhouette_score: ")
print (silhouette_score)